In [1]:
# Import lib
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from functools import wraps
from fuzzywuzzy import fuzz as f_fuzz, process as f_process
from rapidfuzz import fuzz as r_fuzz, process as r_process
import Levenshtein
import ipywidgets


In [2]:
def print_timing(f):
    @wraps(f)
    def timeit(*args, **kwargs):
        t0 = timer()
        try:
            return f(*args, **kwargs)
        finally:
            print(f'\n{f.__name__} finised in {1e3*(timer()-t0):.1f}ms')
        return timeit

In [3]:
df = pd.read_csv('Index_of_Place_Names_in_Great_Britain_(July_2016).csv')
cities = df['place15nm'].unique()

@print_timing
@ipywidgets.interact(input_text='')
def interact_cities(input_text):
    if not input_text: return
    for city, score, _ in f_process.extract(input_text, cities, limit=10):
        print(f'{city:60}{score:3.2f}')

# ipywidgets.interact(interact_cities, input_text='')

interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

In [4]:
f_fuzz.ratio('Tokyo and Oaska', 'Tokio and Osaka')

87

In [5]:
words = ['knight', 'knuth', 'nigh', 'ignite', 'knighthood', 'knead', 'the']
f_process.extract('knight', words)

[('knight', 100),
 ('nigh', 90),
 ('knighthood', 90),
 ('knuth', 55),
 ('ignite', 50)]

### Levenshtein distance with DP (Wagner-Fischer)

In [6]:
def levenshtein_distance(a: str, b:str, verbose:bool=False) -> int:
    m, n = len(a), len(b)
    d = np.zeros((m+1, n+1), dtype=int)

    for i in range(m+1): d[i,0] = i
    for i in range(n+1): d[0,i] = i


    for j in range(1, n+1):
        for i in range(1, m+1):
            cost = 1 if a[i-1] != b[j-1] else 0
            d[i,j] = min(d[i-1, j-1] + cost,     # subsitute 
                         d[i, j-1] + 1,          # delete from B (horizontal)
                         d[i-1, j] + 1)          # insert into B (vertical)

    if verbose: print(d)
    return d[m,n]

In [7]:
d = levenshtein_distance('cat', 'wildcat', verbose=True)
d

[[0 1 2 3 4 5 6 7]
 [1 1 2 3 4 4 5 6]
 [2 2 2 3 4 5 4 5]
 [3 3 3 3 4 5 5 4]]


4

### Comparison of Levenshtein algorithm speed

In [8]:
%timeit Levenshtein.distance('eat', 'rat')
%timeit Levenshtein.distance('knight', 'knigth')
%timeit Levenshtein.distance('unimaginable', 'imagination')

487 ns ± 51 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
530 ns ± 77.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
367 ns ± 7.64 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [9]:
%timeit levenshtein_distance('eat', 'rat')
%timeit levenshtein_distance('knight', 'knigth')
%timeit levenshtein_distance('unimaginable', 'imagination')

12.7 µs ± 779 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
49 µs ± 3.04 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
143 µs ± 15.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Vectorization

In [10]:
def levenshtein_distance_vec(a: np.ndarray, b:np.ndarray) -> np.ndarray:
    m, n, k = len(a), len(b), b.shape[1]
    d = np.zeros((m+1, n+1, k), dtype=np.uint16)   # d[i,j] = levenshtein_distance

    for i in range(n+1): d[i,0] = i
    for i in range(m+1): d[0,i] = i

    for j in range(1,n+1):
        for i in range(1, m+1):
            cost = a[i-1] != b[j-1]
            d[i,j] = np.min([d[i-1] + cost,
                            d[i,j] + 1,
                            d[i-1, j] + 1], axis=0)

    return d[m,n]

In [11]:
len_to_cities = {}
len_to_mat = {}
for w in cities: len_to_cities.setdefault(len(w), []).append(w)
for n, ws in len_to_cities.items():
    for w in ws:
        for c in w:
            len_to_mat[n]=np.asarray(ord(c))

@timer
@ipywidgets.interact(input_text='')
def interact_cities_levenshtein_vec(input_text):
    a_vec = np.asarray([ord(c) for c in input_text])
    n = len(input_text)
    xs, ys = [], []

    for m, b_mat in len_to_mat.items():
        if abs(m-n) > 5: continue
        dist = levenshtein_distance_vec(a_vec, b_mat)
        scores = 1.0 - dist/max(n,m)
        xs.extend(len_to_cities[m])
        ys.extend(scores)

    if not input_text: return
    for score, city in list(sorted(zip(ys,xs), reverse=True))[:10]:
        print(f'{city:60}{score:3.2f}')

interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

TypeError: perf_counter() takes no arguments (1 given)